In [28]:
#imports
import pandas as pd
import numpy as np
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from rake_nltk import Rake
import json

In [29]:
#load camera dataset
#dataframe = pd.read_csv("camera.csv")
#dataframe = pd.read_csv("book.csv")
dataframe = pd.read_csv("tool.csv")
#dataframe

In [30]:
#sentiment analysis
def get_sentiment(comment):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(comment) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    sentiment_score = TextBlob(comment).sentiment.polarity
    if(sentiment_score < -0.1):
        sentiment = "negative"
    elif(sentiment_score > 0.1):
        sentiment = "positive"
    else:
        sentiment = "neutral"
    return [sentiment_score, sentiment]

In [31]:
#keyword extraction
def get_keywords(comment):
    r = Rake()
    r.extract_keywords_from_text(comment)
    keywords = r.get_ranked_phrases()
    return keywords[:10]

In [32]:
#create objects
all_data = dataframe.values

data_list = []

for data in all_data:
    obj = {}
    obj["product_id"] = data[3]
    obj["product_name"] = data[5]
    obj["product_category"] = data[6]
    obj["reviewer_id"] = data[1]
    obj["review_date"] = data[14]
    obj["review_header"] = data[12]
    obj["review_body"] = data[13].replace("<br />", '').replace('\\', '')
    obj["star_rating"] = data[7]
    obj["sentiment_score"] = get_sentiment(data[13].replace("<br />", '').replace('\\', ''))[0]
    obj["sentiment"] = get_sentiment(data[13].replace("<br />", '').replace('\\', ''))[1]
    obj["keywords"] = get_keywords(data[13].replace("<br />", '').replace('\\', ''))
    data_list.append(obj)

data_list

[{'product_id': 'B000JFO1TM',
  'product_name': '- Northern Industrial Tools 245-Pc. Rotary Tool Kit - RP2145CK',
  'product_category': 'Tools',
  'reviewer_id': 14615504,
  'review_date': '6/8/2015',
  'review_header': 'Best tool purchase in a while.',
  'review_body': "I use this constantly. It does everything I want it to and a few things I didn't know a rotary tool could.",
  'star_rating': 5,
  'sentiment_score': -0.1,
  'sentiment': 'neutral',
  'keywords': ['rotary tool could',
   'want',
   'use',
   'things',
   'know',
   'everything',
   'constantly']},
 {'product_id': 'B000JFO1TM',
  'product_name': '- Northern Industrial Tools 245-Pc. Rotary Tool Kit - RP2145CK',
  'product_category': 'Tools',
  'reviewer_id': 47265633,
  'review_date': '4/10/2015',
  'review_header': 'Great buy for the price.',
  'review_body': "I've been using this tool kit for a few months now and it has worked wonderfully. It does what I need it to do. The only problem that I had was that the polishing

In [33]:
#save in json
with open('alfred_data_tool.json', 'w') as fout:
    json.dump(data_list, fout)